In [4]:
import pandas as pd
import os
import re

The part is split into two parts, because originally we just try with close price.

In [2]:
closeprice_10min =[]

for root, directory,files in os.walk('F:\MSc DS\Mini project\DataSet_B02'):
    for filename in files:
        if filename.endswith('csv'):
            filepath =os.path.join(root,filename)
            datacsv = pd.read_csv(filepath,encoding='latin1',names=[0,1,'time','price',4,5,6,7,8,9,10])
            count =1
            for i in range(len(datacsv)-1):
                if datacsv['time'][i] <600* count and datacsv['time'][i+1] >= 600*count:
                    closeprice_10min.append(datacsv['price'][i])
                    count = count+1
            closeprice_10min.append(datacsv['price'][len(datacsv)-1])

import pickle

output = open('.\Dataset_B02_10mins.pkl', 'wb')
pickle.dump(closeprice_10min, output)
output.close()

In [ ]:
# import os
# import re
# import pandas as pd
# pattern =r'(2022.*)(tapes)'
# closeprice_10min =[]
Openprice_10 = []
Highprice_10 = []
Lowprice_10 = []
Volume_10 = []
def extract_OHLCV(filepath):
    data = pd.read_csv(filepath, encoding='latin1', names=[0, 1, 'time', 'price', 4, 5, 6, 7, 'qty', 9, 10])
    data['qty_n'] = 0
    for i in range(len(data)):
        s = data.qty[i]
        data.qty_n[i] = int(re.findall(r".*(\d+)", s)[0])
    data['amount'] = data['price'] * data['qty_n']
    datanew = data.loc[:, ['time','price', 'amount']]

    split_position = []
    count = 1
    for i in range(len(datanew) - 1):
        if datanew['time'][i] < 600 * count and datanew['time'][i + 1] >= 600 * count:
            split_position.append(i)
            count += 1
    split_position.insert(0, -1)
    split_position.append(len(datanew))
    Low = []
    High = []
    Open = []
    Volume = []
    for i in range(len(split_position) - 1):
        Low.append(datanew.loc[split_position[i] + 1:split_position[i + 1] + 1, 'price'].min())
        High.append(datanew.loc[split_position[i] + 1:split_position[i + 1] + 1, 'price'].max())
        # Close = datanew.loc[:split_position[0]+1,'price'][-1]

        Open.append(
            datanew.loc[split_position[i] + 1:split_position[i + 1] + 1, 'price'][split_position[i] + 1])  # not 0
        Volume.append(datanew.loc[split_position[i] + 1:split_position[i + 1] + 1, 'amount'].sum())
    return Open, High, Low, Volume


for root, directory,files in os.walk('F:\MSc DS\Mini project\DataSet_B02'):
    for filename in files:
        if filename.endswith('csv'):
            filepath =os.path.join(root,filename)
            Open, High, Low, Volume = extract_OHLCV(filepath)
            Openprice_10 = Openprice_10 + Open
            Highprice_10 = Highprice_10 + High
            Lowprice_10 = Lowprice_10 + Low
            Volume_10 = Volume_10 + Volume

import pickle
saveddata = open('.\Dataset_B02_10mins.pkl', 'rb')

data2 = pickle.load(saveddata)
Closeprice_10 = data2[:2040] # drop the final file close price there is something wrong
dictprice = {'Open':Openprice_10,'High':Highprice_10,'Low':Lowprice_10,'Close':Closeprice_10,'Volume':Volume_10}
time_interval_10 = pd.DataFrame(dictprice)
time_interval_10.to_pickle('.\Dataset_B02_10mins_OHLCV.pkl')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
